In [ ]:
from shutil import copyfile

copyfile(src = "../input/img2vector/img_to_vec.py", dst = "./img_to_vec.py")

from img_to_vec import *

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
from PIL import Image
from tqdm import tqdm, tqdm_notebook 

from xgboost import XGBRegressor

In [ ]:
TRAIN_PATH = "../input/petfinder-pawpularity-score/train.csv"
TEST_PATH = "../input/petfinder-pawpularity-score/test.csv"
TRAIN_IMAGE_FOLDER = "../input/petfinder-pawpularity-score/train"
TEST_IMAGE_FOLDER = "../input/petfinder-pawpularity-score/test"

SAMPLE_SUBMISSION_PATH = "../input/petfinder-pawpularity-score/sample_submission.csv"
SUBMISSION_PATH = "submission.csv"

ID = "Id"
TARGET = "Pawpularity"
IMAGE_TYPE = 'jpg'
TREE_METHOD = 'gpu_hist'

In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

In [ ]:
def fe_ima2vec(df, img_path, img_col, filename_extension):
    
    img2vec = Img2Vec(cuda=(torch.cuda.is_available()))
    
    fe_img = []
    for idx in tqdm(range(len(df))):
        img = Image.open(f'{img_path}/{df.loc[idx, img_col]}.{filename_extension}')
        vec = img2vec.get_vec(img)
        fe_img.append(vec)
    
    fe_img = pd.DataFrame(fe_img)
    fe_img.columns = ['img_vec_' + str(i) for i in range(fe_img.shape[1])]
    return fe_img

In [ ]:
fe_train = fe_ima2vec(train, TRAIN_IMAGE_FOLDER, ID, IMAGE_TYPE)
fe_train.head()

In [ ]:
fe_test = fe_ima2vec(test, TEST_IMAGE_FOLDER, ID, IMAGE_TYPE)
fe_test.head()

In [ ]:
fe_train.columns

In [ ]:
fe_train.shape,train.shape

In [ ]:
fe_test.shape,test.shape

In [ ]:
new_train = pd.concat([train,fe_train],axis=1)
new_test = pd.concat([test,fe_test],axis=1)
new_train.shape,new_test.shape

In [ ]:
new_train.head()

In [ ]:
y = new_train[TARGET]
X = new_train.drop([ID,TARGET],axis=1)
X_test = new_test.drop([ID],axis=1)

In [ ]:
model = XGBRegressor(tree_method=TREE_METHOD) 
model.fit(X, y)

In [ ]:
pred_test = model.predict(X_test)
pred_test[:5]

In [ ]:
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] =pred_test
submission.to_csv(SUBMISSION_PATH,index=False)
submission.head()